# Coursera Applied Data Science Capston - week 3

### Segmenting and clustering neighborhoods in Toronto

*please note: this Notebook contains all the three parts required for submission*

---

**Part 1: scraping data from Wikipedia and building a Pandas data frame**

---


In [132]:
# importing reguired libraries
import numpy as np
import pandas as pd # data management
import geocoder # geo location
from geopy.geocoders import Nominatim # geo location
import folium # geo visualization
import requests
from sklearn.cluster import KMeans # clustering algorythm
from matplotlib import cm, colors # plotting

In [74]:
# url of Wikepedia page
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
df = pd.read_html(url)

In [75]:
print(type(df), len(df) )

<class 'list'> 3


In [76]:
print(df)

[    Postal Code           Borough  \
0           M1A      Not assigned   
1           M2A      Not assigned   
2           M3A        North York   
3           M4A        North York   
4           M5A  Downtown Toronto   
..          ...               ...   
175         M5Z      Not assigned   
176         M6Z      Not assigned   
177         M7Z      Not assigned   
178         M8Z         Etobicoke   
179         M9Z      Not assigned   

                                         Neighbourhood  
0                                         Not assigned  
1                                         Not assigned  
2                                            Parkwoods  
3                                     Victoria Village  
4                            Regent Park, Harbourfront  
..                                                 ...  
175                                       Not assigned  
176                                       Not assigned  
177                                      

pd.read_html returns a list. The first element is the data frame we need 

In [77]:
df = df[0]

In [78]:
print(type(df))

<class 'pandas.core.frame.DataFrame'>


Yes, it's a Pandas DF

Now, let's clean the Data Frame:
- delete rows where Borough is "Not assigned"

In [79]:
df = df[df['Borough'] != 'Not assigned']
df.shape

(103, 3)

In [80]:
df.head(10)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


Check that Neighbourhood column is always assigned

In [81]:
df[df['Neighbourhood'] == 'Not assigned']

,Postal Code,Borough,Neighbourhood


Ok!

Now, check that postal code is unique

In [82]:
sum((df['Postal Code'].value_counts() > 1))

0

Ok!

Now, check if there are duplicated Neighborhood  

In [83]:
df['Neighbourhood'].value_counts().sort_values(ascending=False)

Downsview                                                                                                                                 4
Don Mills                                                                                                                                 2
Runnymede, Swansea                                                                                                                        1
University of Toronto, Harbord                                                                                                            1
Caledonia-Fairbanks                                                                                                                       1
                                                                                                                                         ..
Parkdale, Roncesvalles                                                                                                                    1
High Park, The Junct

We see that *Downsview* and *Don Mills* are **duplicated**
Let's see these rows

In [84]:
df[(df['Neighbourhood'] == 'Downsview') | (df['Neighbourhood'] == 'Don Mills')]

,Postal Code,Borough,Neighbourhood
11,M3B,North York,Don Mills
20,M3C,North York,Don Mills
65,M3K,North York,Downsview
74,M3L,North York,Downsview
83,M3M,North York,Downsview
92,M3N,North York,Downsview


These two neighbourhoods seem to have different Postal Codes
For the scope of this analyses, we keep them as distinct locations

Print the shape of the DF WITHOUT removing duplicates

In [85]:
df.shape

(103, 3)

---

**Part 2: retrieve geographical coordinates of the Neighbourhoods**

---

**Note: seems that geocoder.google requires authentication** ([see this post](https://stackoverflow.com/questions/55551275/python-geocoder-request-denied-even-with-api-key))

So, I switch to the provided CSV

In [86]:
coords = pd.read_csv('http://cocl.us/Geospatial_data')
coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merging initial data frame with lat, long coords

In [87]:
df = df.merge(coords, how='inner', on='Postal Code')

Check shape

In [88]:
df.shape

(103, 5)

Ok!

Now, check if there are some missing values

In [89]:
df.isnull().sum()

Postal Code      0
Borough          0
Neighbourhood    0
Latitude         0
Longitude        0
dtype: int64

Ok!

And this is our actual DF:

In [90]:
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


---

**Part 3: segmentation and clustering**

---

(Here I follow the steps from the preceeding lab)

Plot a map of Toronto with Markers representing the neighbourhoods

In [91]:
# Toronto coords:
geolocator = Nominatim(user_agent = 'Toronto-explorer')
location = geolocator.geocode('Toronto, Ontario')
latitude = location.latitude
longitude = location.longitude

In [92]:
# base map
base_map = folium.Map(location=[latitude, longitude], zoom_start = 10)
# plot neighbourhoods
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True
    ).add_to(base_map)
    

base_map

Drop Postal Code, as it's no longer needed

In [93]:
df = df.drop(['Postal Code'], axis= 1)

Now, retrieve Neighbourhood information through Foursquare APIs

In [94]:
# define Foursquare credentials
CLIENT_ID = 'ELCGO4JTDN3CY4NULPEECJK2GBOEG5QY3HV2VZBKVQV2JFWM' # your Foursquare ID
CLIENT_SECRET = 'GIZWPHWRTEMWAPRMK0FBATZZI2AXJCOQ54FUPDI05LL0FAO0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

The functions in the next cell are copied from Manhattan Lab

In [105]:
# retrieve venues in Neighbourhood within a specified radius (default = 500)
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# extract category of a venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# most common venue per neighborhood
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]    

In [96]:
# use getNearbyVenues function to retrieve venues for every Neighbourhood and store results in the toronto_venues DF
toronto_venues = getNearbyVenues(names = df['Neighbourhood'], latitudes = df['Latitude'], longitudes = df['Longitude'])

In [97]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [98]:
toronto_venues.shape

(2112, 7)

So we have 2112 venues (with a LIMIT of 100)

Let's see how many venues are there for every Neighbourhood

In [99]:
with pd.option_context('display.max_rows', None):  # more options can be specified also
    print(toronto_venues['Neighborhood'].value_counts())


First Canadian Place, Underground city                                                                                                    100
Toronto Dominion Centre, Design Exchange                                                                                                  100
Commerce Court, Victoria Hotel                                                                                                            100
Garden District, Ryerson                                                                                                                  100
Harbourfront East, Union Station, Toronto Islands                                                                                         100
Stn A PO Boxes                                                                                                                             98
Richmond, Adelaide, King                                                                                                                   93
St. Ja

In [100]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 265 uniques categories.


Now, build a data frame with categories encoded as dummy variable

In [101]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
# watch out: there's a categoty named 'Neighborhood' witch can be confused with the name of the Neighborhood
# this category will be overwritten with the name of the Neighborhood
fixed_columns = list(toronto_onehot.columns[:-1])
fixed_columns.remove("Neighborhood")
fixed_columns = ["Neighborhood"] + fixed_columns
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Now, group back by neighbourhood and calculate mean frequency of every category

In [102]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head(10)

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.0,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0000,0.0000,0.000,0.0000,0.000,0.041667,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Berczy Park,0.0,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,0.0,...,0.0,0.0,0.017857,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,"Birch Cliff, Cliffside West",0.0,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,"Brockton, Parkdale Village, Exhibition Place",0.0,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,"Business reply mail Processing Centre, South C...",0.0,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.0625,0.0625,0.125,0.1875,0.125,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [149]:
toronto_grouped.shape

(96, 264)

*Note: number of rows in the grouped DF (96) is less than original DF*

*This may depend on the fact that some neighborhood has no venues in the Foursquare data base*

Let's print some neighbourhood and relative most frequent category (limit to 10 neighbourhoods)

In [150]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood'][:10]:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant  0.25
1             Breakfast Spot  0.25
2                     Lounge  0.25
3               Skating Rink  0.25
4  Middle Eastern Restaurant  0.00


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.25
1        Pharmacy  0.12
2    Dance Studio  0.12
3  Sandwich Place  0.12
4             Gym  0.12


----Bathurst Manor, Wilson Heights, Downsview North----
              venue  freq
0       Coffee Shop  0.08
1              Bank  0.08
2          Pharmacy  0.04
3     Shopping Mall  0.04
4  Sushi Restaurant  0.04


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1                 Bank  0.25
2   Chinese Restaurant  0.25
3                 Café  0.25
4               Museum  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0         Coffee Shop  0.08
1      Sandwich Place  0.08
2  Italian Restaurant  0.08
3          Hobby Shop  0.

Create a dataframe with the 10 most common category for each neighborhood and print first results

In [151]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Skating Rink,Latin American Restaurant,Breakfast Spot,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
1,"Alderwood, Long Branch",Pizza Place,Dance Studio,Gym,Pharmacy,Coffee Shop,Sandwich Place,Pub,Dog Run,Diner,Discount Store
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Frozen Yogurt Shop,Shopping Mall,Sandwich Place,Restaurant,Diner,Middle Eastern Restaurant,Deli / Bodega,Intersection
3,Bayview Village,Japanese Restaurant,Bank,Chinese Restaurant,Café,Women's Store,Drugstore,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Sandwich Place,Coffee Shop,Thai Restaurant,Indian Restaurant,Café,Sushi Restaurant,Butcher,Juice Bar,Restaurant


### Clustering with k-means

In [152]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_[:10])
print(kmeans.labels_[-11:-1]) 

[0 0 0 0 0 0 0 0 0 0]
[0 4 0 2 0 0 0 0 2 2]


Next cells adds the cluster label to the DF with the 10 most common categories

In [153]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [154]:

toronto_merged = df
toronto_merged.rename({'Neighbourhood':'Neighborhood'}, inplace=True, axis=1)

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood', how='inner')

toronto_merged.head() 

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Parkwoods,43.753259,-79.329656,3,Food & Drink Shop,Park,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store
1,North York,Victoria Village,43.725882,-79.315572,0,Hockey Arena,Pizza Place,Coffee Shop,Portuguese Restaurant,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
2,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Pub,Theater,Shoe Store,Farmers Market,Electronics Store
3,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0,Clothing Store,Furniture / Home Store,Accessories Store,Coffee Shop,Boutique,Miscellaneous Shop,Event Space,Vietnamese Restaurant,Coworking Space,Distribution Center
4,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Park,Diner,Sandwich Place,Smoothie Shop,Burger Joint,Burrito Place,Beer Bar,Café


Plot clusters on map

In [155]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster examination

Cluster 0

In [157]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Victoria Village,Hockey Arena,Pizza Place,Coffee Shop,Portuguese Restaurant,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
2,"Regent Park, Harbourfront",Coffee Shop,Park,Bakery,Breakfast Spot,Café,Pub,Theater,Shoe Store,Farmers Market,Electronics Store
3,"Lawrence Manor, Lawrence Heights",Clothing Store,Furniture / Home Store,Accessories Store,Coffee Shop,Boutique,Miscellaneous Shop,Event Space,Vietnamese Restaurant,Coworking Space,Distribution Center
4,"Queen's Park, Ontario Provincial Government",Coffee Shop,Sushi Restaurant,Park,Diner,Sandwich Place,Smoothie Shop,Burger Joint,Burrito Place,Beer Bar,Café
6,"Malvern, Rouge",Fast Food Restaurant,Women's Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore
...,...,...,...,...,...,...,...,...,...,...,...
98,"The Kingsway, Montgomery Road, Old Mill North",River,Smoke Shop,Women's Store,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
99,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Hotel,Mediterranean Restaurant,Pub,Smoke Shop,Burger Joint
100,"Business reply mail Processing Centre, South C...",Gym / Fitness Center,Garden Center,Park,Pizza Place,Restaurant,Burrito Place,Brewery,Skate Park,Farmers Market,Fast Food Restaurant
101,"Old Mill South, King's Mill Park, Sunnylea, Hu...",Deli / Bodega,Baseball Field,Women's Store,Drugstore,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Diner


Cluster 1

In [158]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,"Rouge Hill, Port Union, Highland Creek",Bar,Women's Store,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


Cluster 2

In [159]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Woodbine Heights,Park,Beer Store,Skating Rink,Curling Ice,Drugstore,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
16,Humewood-Cedarvale,Field,Trail,Hockey Arena,Park,Gourmet Shop,Dessert Shop,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
21,Caledonia-Fairbanks,Park,Bar,Women's Store,Grocery Store,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
35,"East Toronto, Broadview North (Old East York)",Intersection,Park,Convenience Store,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
49,"North Park, Maple Leaf Park, Upwood Park",Trail,Park,Bakery,Construction & Landscaping,Basketball Court,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
61,Lawrence Park,Park,Bus Line,Swim School,Dim Sum Restaurant,Concert Hall,Construction & Landscaping,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store
64,Weston,Park,Jewelry Store,Convenience Store,Women's Store,Drugstore,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
66,York Mills West,Electronics Store,Park,Convenience Store,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store
68,"Forest Hill North & West, Forest Hill Road Park",Trail,Park,Sushi Restaurant,Jewelry Store,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Women's Store
77,"Kingsview Village, St. Phillips, Martin Grove ...",Park,Mobile Phone Shop,Bus Line,Sandwich Place,Donut Shop,Distribution Center,Dog Run,Doner Restaurant,Drugstore,Falafel Restaurant


Cluster 3

In [160]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkwoods,Food & Drink Shop,Park,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store
45,"York Mills, Silver Hills",Park,Women's Store,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant


Cluster 4

In [161]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,"West Deane Park, Princess Gardens, Martin Grov...",Bakery,Women's Store,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
